In [1]:
!pip install datasets evaluate accelerate

In [1]:
from huggingface_hub import notebook_login
notebook_login()
import os
import random
import json
import torch
import torch.nn as nn
from collections import namedtuple
from dataclasses import dataclass , field , asdict
import evaluate
import numpy as np
from datasets import load_dataset, DatasetDict
from transformers import Trainer
from transformers import AutoTokenizer , TrainingArguments
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score



In [2]:
# Load the dataset
imdb = load_dataset('imdb')

In [3]:
# Khởi tạo và tải tokenizer
# tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
# Đặt id của token pad bằng id của token eos trong tokenizer.
tokenizer.pad_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Tạo chức năng tiền xử lý để mã hóa văn bản và cắt bớt các chuỗi không dài hơn độ dài đầu vào tối đa của mã thông báo
# Hàm tiền xử lý dữ liệu
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True, max_length=512)


In [5]:
# Thực hiện mã hóa văn bản
tokenized_imdb = imdb.map(preprocess_function, batched=True)


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [6]:
# Set seed cho hàm random
random.seed(42)

In [7]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer

# Khởi tạo tokenizer và mô hình GPT-2
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)
model.to("cuda")
# Chọn một token từ tokenizer làm token padding
tokenizer.pad_token = tokenizer.eos_token

# Cài đặt token padding cho mô hình
model.config.pad_token_id = tokenizer.pad_token_id



Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Tạo tập train và test
train_dataset = tokenized_imdb["train"]
test_dataset = tokenized_imdb["test"]
# Tạo tập evaluation để đánh giá trong lúc train
# Do số lượng tập test lớn nên chỉ lấy mẫu 1% tập dữ liệu test để đánh giá
total_samples = len(test_dataset)
eval_samples = int(0.1 * total_samples)
eval_indices = random.sample(range(total_samples), eval_samples)
eval_dataset = test_dataset.select(eval_indices)

# Tải module "accuracy" từ thư viện evaluate
accuracy = evaluate.load("accuracy")

In [9]:
#Định nghĩa hàm compute_metrics để tính các độ đo hiệu suất (metrics) cho việc đánh giá model.
# Hàm tính toán các chỉ số đánh giá
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {'accuracy': accuracy_score(p.label_ids, preds)}


In [10]:
# Định nghĩa tên project để log thông tin quá trình train trên wandb # os.environ["WANDB_PROJECT"] = "mamba_tutorial"
# Định nghĩa các tham số train trong class TrainingArguments.
# Cụ thể hơn về các tham số hỗ trợ có thể tham khảo: https:// huggingface.co/docs/transformers/main_classes/trainer
# Cấu hình cho việc huấn luyện
training_args = TrainingArguments(
    output_dir="text_classification_gpt2_1", # Tên folder output learning_rate=5e-5,
    per_device_train_batch_size=4, # Số lượng train sample trên mỗi device
    per_device_eval_batch_size=16, # Số lượng eval sample trên mỗi device
    num_train_epochs=1, # Số epoch train
    warmup_ratio=0.01, # Tỉ lệ tăng dần lr trong giai đoạn warmup
    lr_scheduler_type="cosine", # Loại scheduler để giảm lr
    report_to="none", # "wandb" nếu muốn log kết quả
    evaluation_strategy="steps", # Xác định metric đánh giá sau mỗi số bước
    eval_steps=0.1, # Số bước giữa các đợt đánh giá
    save_strategy="steps", # Xác định khi nào lưu checkpoint
    save_steps=0.1, # Số bước giữa các lần lưu checkpoint
    logging_strategy="steps", # Xác định khi nào in thông tin log
    logging_steps=1, # Số bước giữa các lần in thông tin log
    push_to_hub=True, # Đẩy kết quả lên Hub
    load_best_model_at_end=True, # Load model có kết quả evaluation tốt nhất trong quá trình train
)


In [11]:
# Khởi tạo Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # Giảm kích thước tập dữ liệu để demo
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

In [12]:
# Bắt đầu quá trình train bằng cách gọi hàm train() trên classs trainer.
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
625,0.908800,0.622866,0.662000
1250,1.018800,0.542880,0.709200
1875,0.978100,0.508628,0.755600
2500,0.691900,0.457140,0.792000
3125,0.002000,0.527771,0.789200
3750,1.683400,0.534828,0.810400
4375,0.043600,0.473249,0.826000
5000,0.033200,0.499525,0.825200
5625,0.990700,0.476433,0.834400
6250,0.965100,0.480854,0.836400


TrainOutput(global_step=6250, training_loss=0.5410098761456553, metrics={'train_runtime': 1922.3031, 'train_samples_per_second': 13.005, 'train_steps_per_second': 3.251, 'total_flos': 6532418764800000.0, 'train_loss': 0.5410098761456553, 'epoch': 1.0})

In [13]:
# Đánh giá mô hình trên tập dữ liệu kiểm tra
trainer.evaluate()


{'eval_loss': 0.45714032649993896,
 'eval_accuracy': 0.792,
 'eval_runtime': 42.1372,
 'eval_samples_per_second': 59.33,
 'eval_steps_per_second': 3.726,
 'epoch': 1.0}

In [14]:
# Lưu mô hình
trainer.save_model("./gpt2_for_imdb")

In [16]:
# Đẩy model lên huggingface hub
trainer.push_to_hub(commit_message="Training complete")

CommitInfo(commit_url='https://huggingface.co/badrabbitt/text_classification_gpt2_1/commit/eff55f4b5e3b058c68680280114694e932c807a4', commit_message='Training complete', commit_description='', oid='eff55f4b5e3b058c68680280114694e932c807a4', pr_url=None, pr_revision=None, pr_num=None)

In [17]:
# Thực hiện dự đoán trên tập dữ liệu validation
outputs = trainer.predict(test_dataset)
print(outputs.metrics)

{'test_loss': 0.47162023186683655, 'test_accuracy': 0.78556, 'test_runtime': 434.4293, 'test_samples_per_second': 57.547, 'test_steps_per_second': 3.598}


In [ ]:
!huggingface-cli model info badrabbitt/mamba_text_classification
!pip install transformers

In [38]:
# Tải model Mamba từ model đã được train trước đó.
model = GPT2ForSequenceClassification.from_pretrained("badrabbitt/text_classification_gpt2")
model.to("cuda")
# Tải tokenizer của model Mamba từ model đã được train trước đó.
tokenizer = AutoTokenizer.from_pretrained("badrabbitt/text_classification_gpt2")
# Chọn một token từ tokenizer làm token padding
tokenizer.pad_token = tokenizer.eos_token

# Cài đặt token padding cho mô hình
model.config.pad_token_id = tokenizer.pad_token_id

OSError: Can't load tokenizer for 'badrabbitt/text_classification_gpt2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'badrabbitt/text_classification_gpt2' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.

In [ ]:
id2label = {0: "NEGATIVE", 1: "POSITIVE"}
text = imdb['test'][0]['text']
label = imdb['test'][0]['label']
response = model.predict(text, tokenizer, id2label)
print(f"Classify: {text}\nGT: {id2label[label]}\nPredict: {response}")